# Moving, rotating, mirroring

There are several actions we can take to move and rotate PHIDL objects.  These
actions include movement, rotation, and reflection. There are several types of
PHIDL objects (`Component`, `DeviceReference`, `Port`, `Polygon`, `CellArray`,
`Label`, and `Group`) but they all can be moved and manipulated in the same
ways.

## Basic movement and rotation

We'll start by creating a blank Component and some shapes.  We'll add the shapes
to
the Component as references

Based on phidl tutorial

In [ ]:
import pp
from pp import Component, qp

# Start with a blank Component
D = Component()

# Create some more shape Devices
T = pp.c.text('hello', size = 10, layer = 1)
E = pp.c.ellipse(radii = (10,5))
R = pp.c.rectangle(size = (10,3), layer = 2)

# Add the shapes to D as references
text = D << T
ellipse = D << E
rect1 = D << R
rect2 = D << R

qp(D) # quickplot it!

In [ ]:
c = Component('move_one_ellipse')
e1 = c << pp.c.ellipse(radii = (10,5))
e2 = c << pp.c.ellipse(radii = (10,5))
e1.movex(10) 

print(c)
pp.qp(c)

Now let's practice moving and rotating the objects:

In [ ]:
D = Component('ellipse')
E = pp.c.ellipse(radii = (10, 5))
e1 = D << E
e2 = D << E
qp(D)

In [ ]:
D = Component('ellipse')
E = pp.c.ellipse(radii = (10, 5))
e1 = D << E
e2 = D << E
e2.move(origin = [5, 5], destination = [10, 10]) # Translate by dx = 5, dy = 5
qp(D)

In [ ]:
D = Component('ellipse')
E = pp.c.ellipse(radii = (10,5))
e1 = D << E
e2 = D << E
e2.move([5, 5]) # Translate by dx = 5, dy = 5
qp(D)

In [ ]:
c = Component('rectangles')
R = pp.c.rectangle(size = (10, 5), layer = 1)
rect1 = c << R
rect2 = c << R

rect1.rotate(45) # Rotate the first waveguide by 45 degrees around (0,0)
rect2.rotate(-30, center = [1,1]) # Rotate the second waveguide by -30 degrees around (1,1)

qp(c) 

In [ ]:
c = Component()
text = c << pp.c.text('hello')

text.mirror(p1 = [1, 1], p2 = [1, 3]) # Reflects across the line formed by p1 and p2
qp(c)

In [ ]:
c = Component()
text = c << pp.c.text('hello')
qp(c)

## Working with properties

Each Component and DeviceReference object has several properties which can be
used
to learn information about the object (for instance where it's center coordinate
is).  Several of these properties can actually be used to move the geometry by
assigning them new values.

Available properties are:

- `xmin` / `xmax`: minimum and maximum x-values of all points within the object
- `ymin` / `ymax`: minimum and maximum y-values of all points within the object
- `x`: centerpoint between minimum and maximum x-values of all points within the
object
- `y`: centerpoint between minimum and maximum y-values of all points within the
object
- `bbox`: bounding box (see note below) in format ((xmin,ymin),(xmax,ymax))
- `center`: center of bounding box

In [ ]:
print('bounding box:')
print(text.bbox) # Will print the bounding box of text in terms of [(xmin, ymin), (xmax, ymax)]
print('xsize and ysize:')
print(text.xsize) # Will print the width of text in the x dimension
print(text.ysize) # Will print the height of text in the y dimension
print('center:')
print(text.center) # Gives you the center coordinate of its bounding box
print('xmax')
print(text.xmax) # Gives you the rightmost (+x) edge of the text bounding box

Let's use these properties to manipulate our shapes to arrange them a little
better

In [ ]:
c = Component()
text = c << pp.c.text('hello')
E = pp.c.ellipse(radii = (10, 5))
R = pp.c.rectangle(size = (10, 5), layer = 1)
rect1 = c << R
rect2 = c << R
ellipse = c << E
qp(c)


# First let's center the ellipse
ellipse.center = [0,0] # Move the ellipse such that the bounding box center is at (0,0)

# Next, let's move the text to the left edge of the ellipse
text.y = ellipse.y # Move the text so that its y-center is equal to the y-center of the ellipse
text.xmax = ellipse.xmin # Moves the ellipse so its xmax == the ellipse's xmin

# Align the right edge of the rectangles with the x=0 axis
rect1.xmax = 0
rect2.xmax = 0

# Move the rectangles above and below the ellipse
rect1.ymin = ellipse.ymax + 5
rect2.ymax = ellipse.ymin - 5

qp(c)

In addition to working with the properties of the references inside the
Component,
we can also manipulate the whole Component if we want.  Let's try mirroring the
whole Component `D`:

In [ ]:
print(c.xmax) # Prints out '10.0'

c.mirror((0,1)) # Mirror across line made by (0,0) and (0,1)

qp(c)

### A note about bounding boxes

When we talk about bounding boxes, we mean it in the sense of the smallest
enclosing box which contains all points of the geometry.  So the bounding box
for the Component D looks like this:

In [ ]:
# The pp.c library has a handy bounding-box function
# which takes a bounding box and returns the rectangle points for it
import pp
from pp import qp

D = pp.c.text('hi')
bbox = D.bbox
D << pp.c.bbox(bbox=bbox, layer=(2, 0))
qp(D)

In [ ]:
# pp.get_padding_points can also add a bbox with respect to the bounding box edges
D = pp.c.text('bye')
device_bbox = D.bbox
D.add_polygon(pp.get_padding_points(D, padding=1), layer=(2, 0))
qp(D)

When we query the properties of D, they will be calculated with respect to this
bounding-rectangle.  For instance:

In [ ]:
print('Center of Component D:')
print(D.center)

print('X-max of Component D:')
print(D.xmax)

In [ ]:
D = Component()
R = pp.c.rectangle(size = (10,3), layer = 2)
rect1 = D << R
pp.qp(D)

## Chaining commands

Many of the movement/manipulation functions return the object they manipulate.
We can use this to chain multiple commands in a single line.

For instance these two expressions:

In [ ]:
rect1.rotate(angle = 37)
rect1.move([10,20])
pp.qp(D)

...are equivalent to this single-line expression

In [ ]:
D = Component()
R = pp.c.rectangle(size = (10,3), layer = 2)
rect1 = D << R

rect1.rotate(angle = 37).move([10,20])
pp.qp(D)